In [3]:
##  Extractive QA using RoBerTa model ..
from models import gen_with_gemini, model_1
from transformers import AutoModelForQuestionAnswering, AutoTokenizer, pipeline

##  Loading the model ..
model_name = "deepset/roberta-base-squad2"
nlp = pipeline('question-answering', model=model_name, tokenizer=model_name)

#  Extracting the answers ..
def extract_answer(question, context):
    nlp = pipeline('question-answering', model=model_name, tokenizer=model_name)
    QA_input = {"question": question, "context": context}
    res = nlp(QA_input)
    return res['score'], res['answer']


KeyboardInterrupt: 

In [ ]:
import pandas as pd
from tqdm import tqdm

# Initialize lists to store scores and answers
scores = []
answers = []

df = pd.read_csv("valid_invalid.csv")

# Iterate over the first 2 rows of the DataFrame with tqdm progress bar
for _, item in tqdm(df.iterrows(), total=df.shape[0], desc="Processing rows"):
    # Extract question and context from the current row
    question = item['Question']
    context = item['paragraphs']
    
    # Call extract_answer function to obtain score and answer
    score, ans = extract_answer(question, context)
    
    # Append score and answer to lists
    scores.append(score)
    answers.append(ans)

df['Extarctive Answer'] = answers
df['Extraction score'] = scores
df

Processing rows: 100%|██████████| 106/106 [01:56<00:00,  1.10s/it]


In [ ]:
df.columns

Index(['Unnamed: 0', 'Bloom's Taxonomy Category', 'Question', 'validity',
       'Extarctive Answer', 'Extraction score',
       'gemini_pred for valid/invalid', 'gemini_answers',
       'Human Score for correct ans per bloom cat', 'paragraphs', 'Unnamed: 9',
       'Unnamed: 10', 'Unnamed: 11', 'gemini validity'],
      dtype='object')

In [ ]:
df = pd.read_csv("valid_final.csv")
df = df[df['validity'] == "Invalid"]
df = df[df["Bloom's Taxonomy Category"] == "Knowledge (Remembering)"]
df

,Bloom's Taxonomy Category,Question,validity,Extarctive Answer,Extraction score,gemini_pred for valid/invalid,gemini_answers,Human Score for correct ans per bloom cat,paragraphs,Unnamed: 9,Unnamed: 10,Unnamed: 11
12,Knowledge (Remembering),Where did the narrator's mom go for her vacation?,Invalid,Aunt Cathy,0.011716,Delaware,NaN,NaN,The biggest thorn in the paw of my grand plan ...,NaN,NaN,NaN
30,Knowledge (Remembering),What time did the accident occur?,Invalid,Two a.m.,0.028338,The paragraph does not contain the answer to t...,NaN,NaN,Two a.m. and we are standing on the side of th...,NaN,NaN,NaN
52,Knowledge (Remembering),What time did the young man start feeling his ...,Invalid,late at night,0.455681,"Yes, the young man started feeling his breathi...",NaN,NaN,The staffing of ambulances at the moment is .....,NaN,NaN,NaN
74,Knowledge (Remembering),What type of rash did the baby with possible m...,Invalid,right,0.266350,"Yes, the baby had a rash on the back of the knee.",NaN,NaN,I think I'm going to be leaving my job soon.\n...,NaN,NaN,NaN
96,Knowledge (Remembering),What type of weather was it during the incident?,Invalid,crappy,0.465675,"Yes, the weather was cold, dark, and raining.",NaN,NaN,"It was cold, it was dark and it was raining th...",NaN,NaN,NaN


In [ ]:
# Descriminating question for valid or invalid  with gemini ..
from tqdm import tqdm

def generate_answer_prompt(question, paragraph):
    # Construct the prompt with instructions and placeholders
    prompt = (
        f"Instructions: Please read the paragraph below.\n" 
        f"Paragraph: {paragraph}\n"
	    f"Instructions: Now tell with a simple yes or no answer, whether you can answer the question based on the information in the paragraph above.\n"
        f"Question: {question}\n"
        f"Answer:"
        f"Instructions: If answer is Yes, please answer this question {question} from paragraph"
    )
    return prompt

df = pd.read_csv("valid_final.csv")
df = df[df['validity'] == "Invalid"]
df = df[df["Bloom's Taxonomy Category"] == "Knowledge (Remembering)"]


# Iterate over the first 2 rows of the DataFrame with tqdm progress bar
answers = []
for _, item in tqdm(df.iterrows(), total=df.shape[0], desc="Processing rows", leave=True):
    # Extract question and context from the current row
    question = item['Question']
    context = item['paragraphs']
    prompt = generate_answer_prompt(question, context)
    out = gen_with_gemini(prompt, model=model_1)
    print("Question : ", question)
    print("Context : ", context)
    print(out)
    print('-------')
    answers.append(out)

#df['gemini validity'] = answers
#df.to_csv("valid.csv")

Processing rows:   0%|          | 0/5 [00:00<?, ?it/s]

Processing rows:  20%|██        | 1/5 [00:01<00:06,  1.52s/it]

Question :  Where did the narrator's mom go for her vacation?
Context :  The biggest thorn in the paw of my grand plan to stop going to school was that my mom would probably notice that I wasn't going to school.
It wasn't an issue, however. When I got home my mom was running around the house shuttling clothes from closets and drawers to open suitcases.
"Are we going somewhere?" I asked, setting my car keys on the kitchen table.
She stopped in place and stood up, looking at me for a moment.
"I am," she started. "I've been talking to Aunt Cathy on the phone and I don't think she's doing very well, with the divorce and then your father. I'm going to spend a few days out there with her until she's feeling better."
Cathy is my dad's sister and I always suspected her to be a bit batty. She didn't come from Delaware for my dad's, her brother's, funeral; said she wouldn't be able to drive herself, and nobody could pick her up.
My mom continued, "Would that be alright? I mean, will you be okay 

Processing rows:  40%|████      | 2/5 [00:02<00:04,  1.40s/it]

Question :  What time did the accident occur?
Context :  Two a.m. and we are standing on the side of the road waiting for the fire service to take the top off the car in front of us. The wind whistles across the flats making us all shiver despite our fleeces and our jackets.
Two cars have been involved in a high-speed road traffic accident (RTA), the parked car that was hit has been shunted forward leaving ten-yard-long skid marks. The cars aren't too damaged but the seats inside have shifted around, trapping the occupants.
There are seven ambulances here, four fire trucks, half a dozen police and three ambulance officers with clipboards. There are eight patients, all but one need cutting from the cars and collaring and boarding. The only woman involved is 'walking wounded'.
The reason that it is taking so long for our car to get its lid removed by the fire service is because of the position of one of the patients inside. He looks rather unwell and the crew looking after him really wou

...

Question :  What time did the young man start feeling his breathing getting tight?
Context :  The staffing of ambulances at the moment is ... to put it bluntly ... poor.
Working on the fast response unit (FRU - a car that is designed to get to the sickest patients quickly) means that I often get to an emergency call within minutes of it being made. Unfortunately, with so few ambulances on the road, the patient and I are often left staring at each other for long periods of time; in a couple of cases up to an hour.
I was sent to a young man having an asthma attack. It was late at night, and he had been queuing to get into a nightclub when he had started to feel his breathing getting tight, the sign of an asthma attack, so had headed to a taxi office in order to go home. Unfortunatel, his asthma progressed and so an ambulance was called. What he got was me, on my own, in a car.
After dealing with the drunken group of teenage girls that had taken time out of waiting for a cab to start loud

Processing rows:  80%|████████  | 4/5 [00:05<00:01,  1.44s/it]

Question :  What type of rash did the baby with possible meningitis have?
Context :  I think I'm going to be leaving my job soon.
I went to a six-month-old baby with possible meningitis. The baby had the right sort of rash (although it was only on the back of the knee and, after checking, nowhere else). It had a temperature, but was one of the happiest, most alert children I've ever had the pleasure of meeting. It just didn't seem as if it had meningitis, and trust me, I've seen a fair few children and adults with it so I have a pretty good idea what it looks like.
Then, as in the past, I was left waiting for an ambulance.
For around 45 minutes.
There was no way I was going to be able to transport the patient in my car. It's just not equipped to carry such a small child. We don't have baby seats and as the family didn't have a car they didn't have one either.
So the family ended up phoning a friend to take them to the hospital. The ambulance turned up just as they were getting into the

Processing rows: 100%|██████████| 5/5 [00:07<00:00,  1.52s/it]

Question :  What type of weather was it during the incident?
Context :  It was cold, it was dark and it was raining the sort of thin greasy rain that soaks straight through your clothes. I was making my way to one of the Docklands Light Railway stations for a 'Male - collapsed, caller not willing to approach patient.' I'd been to this station in the last week for a hoax call and I wasn't sure if this was a repeat performance.
At the bottom of the stairs just sheltered from the rain was a young man in his twenties, dirty, dressed in filthy clothes and curled up next to a plastic bag. Standing over him was another man, this one dressed in a suit, looking a bit concerned.
(The London borough of Tower Hamlets has both the richest, and the poorest population in London.)
'He's just laying there, not talking,' the smartly dressed man said. 'I didn't really know what to do ...'
I let him know that I'd take care of the patient, and that he had done the right thing and could go home.
It was just

In [ ]:
# Descriminating question for valid or invalid  with gemini ..
from tqdm import tqdm

def generate_answer_prompt(question, paragraph):
    # Construct the prompt with instructions and placeholders
    prompt = (
        f"Instructions: Please read the paragraph below.\n" 
        f"Paragraph: {paragraph}\n"
	f"Instructions: Now tell with a simple yes or no answer, whether you can answer the question based on the information in the paragraph above.\n"
        f"Question: {question}\n"
        
        f"Answer:"
    )
    return prompt

df = pd.read_csv("valid_final.csv")
df = df[df["Bloom's Taxonomy Category"] == 'Understanding']


# Iterate over the first 2 rows of the DataFrame with tqdm progress bar
answers = []
for _, item in tqdm(df.iterrows(), total=df.shape[0], desc="Processing rows", leave=True):
    # Extract question and context from the current row
    question = item['Question']
    context = item['paragraphs']
    prompt = generate_answer_prompt(question, context)
    out = gen_with_gemini(prompt, model=model_1)
    print(out)
    answers.append(out)

#df['gemini validity'] = answers
#df.to_csv("valid.csv")

Processing rows:  10%|█         | 1/10 [00:01<00:13,  1.48s/it]

Yes


Processing rows:  20%|██        | 2/10 [00:02<00:10,  1.33s/it]

Yes


Processing rows:  30%|███       | 3/10 [00:04<00:10,  1.53s/it]

...

Yes


Processing rows:  90%|█████████ | 9/10 [00:12<00:01,  1.36s/it]

Yes


Processing rows: 100%|██████████| 10/10 [00:14<00:00,  1.48s/it]

Yes


In [ ]:
df = pd.read_csv("valid_final.csv")
df = df[df["Bloom's Taxonomy Category"] == 'Understanding']
df = df[df["validity"]== "Invalid"]
df.shape

(0, 12)

In [ ]:
import pandas as pd

df = pd.read_csv('valid.csv')
df['gemini validity'].value_counts()

gemini validity
no     30
Yes    12
yes    11
No      7
Name: count, dtype: int64

In [ ]:
for item in df.iterrows():
    print(item[1]['gemini_pred for valid/invalid'])

Yes, the narrator set their car keys down on the kitchen table upon entering the house.
Yes, the reason Cathy gave for not attending the narrator's father's funeral was that she wouldn't be able to drive herself, and nobody could pick her up.
Yes, the narrator thinks their mom is going to visit their Aunt Cathy because their mom tells them that she has been talking to Aunt Cathy on the phone and that Aunt Cathy is not doing very well, with the divorce and then the narrator's father.
Yes, the phrase "a bit batty" suggests that the narrator believes their Aunt Cathy is slightly crazy or eccentric.
Yes, there is an answer to the question. The narrator's freedom to skip school could negatively impact their grades. Without the structure and accountability of attending school, the narrator may find it difficult to stay on top of their studies and complete assignments. Additionally, the narrator may become distracted by other activities and lose motivation to focus on their schoolwork.
The pa

In [ ]:
from langchain_community.embeddings import HuggingFaceBgeEmbeddings
from sklearn.metrics.pairwise import cosine_similarity
from tqdm import tqdm  # Import tqdm for progress tracking
import numpy as np

chatgpt_scores = []

# Assuming `df` is your DataFrame containing 'gemini_answers' and 'paragraphs'
for _, item in tqdm(df.iterrows(), total=len(df), desc="Calculating Gemini Scores"):
    candidate = item['llama2']
    reference = item['paragraphs']

    model_kwargs = {"device": "cuda"}
    encode_kwargs = {"normalize_embeddings": True}
    hf = HuggingFaceBgeEmbeddings(model_name="BAAI/bge-m3", model_kwargs=model_kwargs, encode_kwargs=encode_kwargs)    
    
    embedding1 = np.array(hf.embed_query(candidate)).reshape(1, -1)
    embedding2 = np.array(hf.embed_query(reference)).reshape(1, -1)
  
    # Calculate cosine similarity between the embeddings
    score = cosine_similarity(embedding1, embedding2)
    print(score)
    chatgpt_scores.append(score)

In [ ]:
prompt = "Explain Taylors series in detail and provide output in HTMl format."



In [ ]:
from models import gen_with_gemini, model_1

out = gen_with_gemini(prompt, model=model_1)
print(out)

## Taylor Series

The Taylor series is a mathematical expansion that represents a function as an infinite sum of terms. It is named after the English mathematician Brook Taylor, who first published the series in 1715.

The Taylor series for a function \(f(x)\) at a point \(x = a\) is given by:

$$f(x) = f(a) + f'(a)(x-a) + \frac{f''(a)}{2!}(x-a)^2 + \frac{f'''(a)}{3!}(x-a)^3 + \cdots$$

where \(f'(a)\), \(f''(a)\), \(f'''(a)\), \(\cdots\) are the first, second, third, \(\cdots\) derivatives of \(f(x)\) at \(x = a\), and \(n!\) is the factorial of \(n\).

The Taylor series can be used to approximate the value of a function at a point \(x\) that is close to \(a\). The more terms that are used in the approximation, the more accurate the approximation will be.

### Output in HTML format

```html
<!DOCTYPE html>
<html>
<head>
  <title>Taylor Series</title>
</head>
<body>
  <h1>Taylor Series</h1>

  <p>The Taylor series is a mathematical expansion that represents a function as an infinite su

In [2]:
###   Some new works ..
context = """Title: The Unlikely Friendship

Once upon a time, in the heart of the Amazon rainforest, lived a wild jaguar named Jago. Jago was known for his sleek coat, piercing green eyes, and his powerful roar that echoed through the forest. He was a solitary creature, as most jaguars are, and he liked it that way.

One day, while hunting for food, Jago stumbled upon a small, injured monkey. The monkey had fallen from a tree and hurt his leg, leaving him unable to move. Jago, being a wild animal, saw the monkey as an easy prey. But as he approached the helpless creature, he saw the fear in the monkey's eyes and something in him changed. He couldn't bring himself to harm the small creature.

Days turned into weeks, and Jago found himself bringing food to the injured monkey. He would leave it near the tree where the monkey was trapped, always keeping a safe distance. The monkey, who the jaguar had started to think of as 'Momo', began to heal and grow stronger.

One day, Momo was finally able to move around. He was still weak, but he was no longer trapped. To Jago's surprise, Momo didn't run away. Instead, he approached Jago slowly, with caution but without fear. From that day forward, an unlikely friendship blossomed between the wild jaguar and the small monkey.

They spent their days exploring the forest together, with Jago protecting Momo from danger and Momo helping Jago find the best fruits in the trees. They became inseparable, and their bond became a legend in the Amazon rainforest.

Their story is a reminder that even in the wild, where survival is the primary instinct, there is room for compassion and friendship. It shows that even the most unlikely of pairs can form a bond, and that every creature has a capacity for kindness, no matter how wild they may seem."""

valids = [
    "Where did the story of Jago and Momo take place?",
    "What kind of animal was Jago?",
    "What was Jago known for?",
    "How did Jago feel about being a solitary creature?",
    "What did Jago stumble upon while hunting for food?",
    "Why was the monkey unable to move?",
    "What did Jago initially see the monkey as?",
    "What stopped Jago from harming the monkey?",
    "What did Jago do for the injured monkey over time?",
    "What name did Jago start to think of the monkey as?",
    "What happened when Momo was finally able to move around?",
    "How did Momo approach Jago?",
    "What kind of friendship blossomed between Jago and Momo?",
    "What did Jago and Momo do together in the forest?",
    "What did their bond become in the Amazon rainforest?",
    "What is the story a reminder of?",
    "What does the story show about the capacity for kindness?",
    "What is the primary instinct in the wild according to the story?",
    "What is the most unlikely of pairs according to the story?",
    "What is the moral of the story?"
]

invalid_outs = [
    "What is the capital of France?",
    "Who was the first president of the United States?",
    "What is the largest planet in our solar system?",
    "What is the chemical symbol for hydrogen?",
    "Who wrote the novel 'Pride and Prejudice'?",
    "What is the square root of 64?",
    "What is the currency of Japan?",
    "Who painted the Mona Lisa?",
    "What is the tallest mountain in the world?",
    "What is the formula for water?",
    "Who discovered penicillin?",
    "What is the largest ocean in the world?",
    "What is the name of the famous scientist who developed the theory of general relativity?",
    "What is the capital city of Australia?",
    "What is the chemical symbol for gold?",
    "Who composed the symphony 'The Four Seasons'?",
    "What is the largest desert in the world?",
    "What is the name of the process by which plants make food?",
    "Who was the first person to walk on the moon?",
    "What is the largest mammal in the world?"
]

invalids = [
    "What was the name of the forest where Jago and Momo lived?",
    "What color were Jago's eyes?",
    "How long had Jago been living in the Amazon rainforest?",
    "What kind of injury did the monkey have?",
    "What was Jago's initial reaction to seeing the monkey?",
    "How many days did Jago bring food to the injured monkey?",
    "What kind of food did Jago bring to the monkey?",
    "What other animals lived in the Amazon rainforest?",
    "What was Momo's reaction when he first saw Jago?",
    "How did Jago and Momo communicate with each other?",
    "What kind of fruits did Momo help Jago find in the trees?",
    "What dangers did Jago protect Momo from in the forest?",
    "What other unlikely friendships existed in the Amazon rainforest?",
    "What was the weather like in the Amazon rainforest?",
    "What was the size of the territory that Jago and Momo roamed?",
    "What was the reaction of other animals in the forest to Jago and Momo's friendship?",
    "What kind of trees did Momo like to climb?",
    "What was the name of the river that flowed through the Amazon rainforest?",
    "What was the average lifespan of a jaguar in the wild?",
    "What was the legend that the animals of the Amazon rainforest told about Jago and Momo's friendship?"
]


In [ ]:
import pandas as pd
from tqdm import tqdm

##  Extractive QA using RoBerTa model ..
from models import gen_with_gemini, model_1
from transformers import AutoModelForQuestionAnswering, AutoTokenizer, pipeline

##  Loading the model ..
model_name = "deepset/roberta-base-squad2"
nlp = pipeline('question-answering', model=model_name, tokenizer=model_name)

#  Extracting the answers ..
def extract_answer(question, context):
    nlp = pipeline('question-answering', model=model_name, tokenizer=model_name)
    QA_input = {"question": question, "context": context}
    res = nlp(QA_input)
    return res['score'], res['answer']

# Initialize lists to store scores and answers
scores = []
answers = []

# Iterate over the first 2 rows of the DataFrame with tqdm progress bar
for question in tqdm(invalid_outs, total=len(invalid_outs), desc="Processing questions"):
    
    # Call extract_answer function to obtain score and answer
    score, ans = extract_answer(question, context)
    
    # Append score and answer to lists
    scores.append(score)
    answers.append(ans)

KeyboardInterrupt: 

In [ ]:
# Create a Pandas DataFrame from the dictionary
df1 = pd.DataFrame({"Questions": valids, "ext_ansers":answers, "ext_scores":scores})
df1

,Questions,ext_ansers,ext_scores
0,Where did the story of Jago and Momo take place?,Amazon rainforest,0.468475
1,What kind of animal was Jago?,solitary,0.140716
2,What was Jago known for?,"his sleek coat, piercing green eyes, and his p...",0.071314
3,How did Jago feel about being a solitary creat...,he liked it that way,0.477561
4,What did Jago stumble upon while hunting for f...,"a small, injured monkey",0.530825
5,Why was the monkey unable to move?,The monkey had fallen from a tree and hurt his...,0.257095
6,What did Jago initially see the monkey as?,easy prey,0.342295
7,What stopped Jago from harming the monkey?,fear,0.041972
8,What did Jago do for the injured monkey over t...,bringing food,0.196375
9,What name did Jago start to think of the monke...,Momo,0.877823


In [ ]:
# Create a Pandas DataFrame from the dictionary
df2 = pd.DataFrame({"Questions": invalids, "ext_ansers":answers, "ext_scores":scores})
df2

,Questions,ext_ansers,ext_scores
0,What was the name of the forest where Jago and...,survival,0.059293
1,What color were Jago's eyes?,green,0.882257
2,How long had Jago been living in the Amazon ra...,survival,0.004552
3,What kind of injury did the monkey have?,leg,0.218828
4,What was Jago's initial reaction to seeing the...,Momo didn't run away,0.110756
5,How many days did Jago bring food to the injur...,weeks,0.274863
6,What kind of food did Jago bring to the monkey?,fruits,0.639673
7,What other animals lived in the Amazon rainfor...,small monkey,0.017332
8,What was Momo's reaction when he first saw Jago?,Momo didn't run away,0.135215
9,How did Jago and Momo communicate with each ot...,slowly,0.067103


In [ ]:
# Create a Pandas DataFrame from the dictionary
df3 = pd.DataFrame({"Questions": invalid_outs, "ext_ansers":answers, "ext_scores":scores})
df3

,Questions,ext_ansers,ext_scores
0,What is the capital of France?,blossomed between the wild jaguar and the smal...,0.000088
1,Who was the first president of the United States?,blossomed between the wild jaguar and the smal...,0.000425
2,What is the largest planet in our solar system?,blossomed between the wild jaguar and the smal...,0.000676
3,What is the chemical symbol for hydrogen?,blossomed between the wild jaguar and the smal...,0.000086
4,Who wrote the novel 'Pride and Prejudice'?,survival,0.003925
5,What is the square root of 64?,blossomed between the wild jaguar and the smal...,0.000020
6,What is the currency of Japan?,blossomed between the wild jaguar and the smal...,0.000374
7,Who painted the Mona Lisa?,blossomed between the wild jaguar and the smal...,0.000086
8,What is the tallest mountain in the world?,blossomed between the wild jaguar and the smal...,0.000406
9,What is the formula for water?,fruits,0.006315


In [ ]:
df = pd.concat([df1, df2, df3])
df.to_csv("new.csv")

In [1]:
import pandas as pd
from models import gen_with_gemini, model_1

df = pd.read_csv("new_data.csv")
df

,Questions,ext_ansers,ext_scores,gemini_answers,mistral_ans
0,Where did the story of Jago and Momo take place?,Amazon rainforest,0.468475,Answer: Amazon rainforest,The story of Jago and Momo took place in the h...
1,What kind of animal was Jago?,solitary,0.140716,Answer: Jaguar,Jago was a wild jaguar.
2,What was Jago known for?,"his sleek coat, piercing green eyes, and his p...",0.071314,Question: What was Jago known for?\nAnswer: An...,"Jago was known for his sleek coat, piercing gr..."
3,How did Jago feel about being a solitary creat...,he liked it that way,0.477561,Answer: Answer doesn't exist in the paragraph.,"Jago liked being a solitary creature, as most ..."
4,What did Jago stumble upon while hunting for f...,"a small, injured monkey",0.530825,Answer: An injured monkey,"Jago stumbled upon a small, injured monkey whi..."
5,Why was the monkey unable to move?,The monkey had fallen from a tree and hurt his...,0.257095,Answer: Answer doesn't exist in the paragraph.,The monkey was unable to move because he had f...
6,What did Jago initially see the monkey as?,easy prey,0.342295,Answer: Easy prey,Jago initially saw the monkey as an easy prey.
7,What stopped Jago from harming the monkey?,fear,0.041972,Answer: Answer doesn't exist in the paragraph.,Jago saw the fear in the monkey's eyes and som...
8,What did Jago do for the injured monkey over t...,bringing food,0.196375,Answer: Answer doesn't exist in the paragraph.,Jago brought food to the injured monkey over t...
9,What name did Jago start to think of the monke...,Momo,0.877823,Answer: Momo,Jago started to think of the monkey as 'Momo'.


In [5]:
from models import gen_with_gemini, model_1
from tqdm import tqdm

def generate_answer_prompt(question, paragraph=context):
    # Construct the prompt with instructions and placeholders
    # Construct the prompt with instructions and placeholders
    prompt = (
        f"Instructions: Please read the paragraph below carefully.\n"
        f"Paragraph: {paragraph}\n"
        f"Instructions: Now determine whether you can answer the question based on the information in the paragraph above. Provide the answer as a yes or no \n"
        f"Question: {question}\n"
        f"Answer: "
    )
    # Check if the answer can be found in the paragraph
    if question.lower() in paragraph:
        prompt += f"The answer is: "
    else:
        prompt += "Answer doesn't exist in the paragraph.\n"
    return prompt

df = pd.read_csv("new.csv")
answers = []
for question in tqdm(df.iterrows(), total=df.shape[0]):
    question = question[1]['Questions']
    prompt = generate_answer_prompt(question)
    out = gen_with_gemini(prompt, model=model_1)
    print(question)
    print(out)
    print('--------')
    answers.append(out)

df['gemini_answers'] = answers
df

  2%|▏         | 1/60 [00:01<01:08,  1.16s/it]

Where did the story of Jago and Momo take place?
Yes
--------


  3%|▎         | 2/60 [00:02<01:14,  1.28s/it]

What kind of animal was Jago?
Answer: yes
--------


  5%|▌         | 3/60 [00:03<01:11,  1.25s/it]

...

 98%|█████████▊| 59/60 [01:17<00:01,  1.33s/it]

Who was the first person to walk on the moon?
Answer: no
--------


100%|██████████| 60/60 [01:18<00:00,  1.31s/it]

What is the largest mammal in the world?
Answer: no
--------


,Unnamed: 0,Questions,ext_ansers,ext_scores,gemini_answers
0,0,Where did the story of Jago and Momo take place?,Amazon rainforest,0.468475,Yes
1,1,What kind of animal was Jago?,solitary,0.140716,Answer: yes
2,2,What was Jago known for?,"his sleek coat, piercing green eyes, and his p...",0.071314,Answer: yes
3,3,How did Jago feel about being a solitary creat...,he liked it that way,0.477561,Answer: Answer doesn't exist in the paragraph.
4,4,What did Jago stumble upon while hunting for f...,"a small, injured monkey",0.530825,Answer: yes
5,5,Why was the monkey unable to move?,The monkey had fallen from a tree and hurt his...,0.257095,Answer: yes
6,6,What did Jago initially see the monkey as?,easy prey,0.342295,Answer: yes
7,7,What stopped Jago from harming the monkey?,fear,0.041972,Answer: Answer doesn't exist in the paragraph.
8,8,What did Jago do for the injured monkey over t...,bringing food,0.196375,Answer: yes
9,9,What name did Jago start to think of the monke...,Momo,0.877823,Answer: yes


In [ ]:
df['mistral_ans'] = mistral_ans

In [ ]:
mistral =[
{
"question": "What is the capital of France?",
"answer": "Not available in the given context."
},
{
"question": "Who was the first president of the United States?",
"answer": "Not available in the given context."
},
{
"question": "What is the largest planet in our solar system?",
"answer": "Not available in the given context."
},
{
"question": "What is the chemical symbol for hydrogen?",
"answer": "Not available in the given context."
},
{
"question": "Who wrote the novel 'Pride and Prejudice'?",
"answer": "Not available in the given context."
},
{
"question": "What is the square root of 64?",
"answer": "Not available in the given context."
},
{
"question": "What is the currency of Japan?",
"answer": "Not available in the given context."
},
{
"question": "Who painted the Mona Lisa?",
"answer": "Not available in the given context."
},
{
"question": "What is the tallest mountain in the world?",
"answer": "Not available in the given context."
},
{
"question": "What is the formula for water?",
"answer": "Not available in the given context."
},
{
"question": "Who discovered penicillin?",
"answer": "Not available in the given context."
},
{
"question": "What is the largest ocean in the world?",
"answer": "Not available in the given context."
},
{
"question": "What is the name of the famous scientist who developed the theory of general relativity?",
"answer": "Not available in the given context."
},
{
"question": "What is the capital city of Australia?",
"answer": "Not available in the given context."
},
{
"question": "What is the chemical symbol for gold?",
"answer": "Not available in the given context."
},
{
"question": "Who composed the symphony 'The Four Seasons'?",
"answer": "Not available in the given context."
},
{
"question": "What is the largest desert in the world?",
"answer": "Not available in the given context."
},
{
"question": "What is the name of the process by which plants make food?",
"answer": "Not available in the given context."
},
{
"question": "Who was the first person to walk on the moon?",
"answer": "Not available in the given context."
},
{
"question": "What is the largest mammal in the world?",
"answer": "Not available in the given context."
}
]


In [ ]:
mistral_ans = ['The story of Jago and Momo took place in the heart of the Amazon rainforest.',
 'Jago was a wild jaguar.',
 'Jago was known for his sleek coat, piercing green eyes, and powerful roar that echoed through the forest.',
 'Jago liked being a solitary creature, as most jaguars are.',
 'Jago stumbled upon a small, injured monkey while hunting for food.',
 'The monkey was unable to move because he had fallen from a tree and hurt his leg.',
 'Jago initially saw the monkey as an easy prey.',
 "Jago saw the fear in the monkey's eyes and something in him changed, so he couldn't bring himself to harm the small creature.",
 'Jago brought food to the injured monkey over time, leaving it near the tree where the monkey was trapped.',
 "Jago started to think of the monkey as 'Momo'.",
 "When Momo was finally able to move around, he didn't run away. Instead, he approached Jago slowly, with caution but without fear.",
 'Momo approached Jago slowly, with caution but without fear.',
 'An unlikely friendship blossomed between Jago and Momo.',
 'Jago and Momo spent their days exploring the forest together, with Jago protecting Momo from danger and Momo helping Jago find the best fruits in the trees.',
 'Their bond became a legend in the Amazon rainforest.',
 'The story is a reminder that even in the wild, where survival is the primary instinct, there is room for compassion and friendship.',
 'The story shows that every creature has a capacity for kindness, no matter how wild they may seem.',
 'The primary instinct in the wild according to the story is survival.',
 'The most unlikely of pairs according to the story is the friendship between a wild jaguar and a small monkey.',
 'The moral of the story is that even the most unlikely of pairs can form a bond, and that every creature has a capacity for kindness.',
 'Not available in the paragraph',
 'Jago was known for his piercing green eyes',
 'Not available in the paragraph',
 'The monkey had fallen from a tree and hurt his leg',
 'Jago saw the monkey as an easy prey',
 'Not available in the paragraph',
 'Not specified in the paragraph',
 'Not specified in the paragraph',
 'Not available in the paragraph',
 'Not specified in the paragraph',
 'Not specified in the paragraph',
 'Not specified in the paragraph',
 'Not specified in the paragraph',
 'Not specified in the paragraph',
 'Not specified in the paragraph',
 'Not specified in the paragraph',
 'Not specified in the paragraph',
 'Not specified in the paragraph',
 'Not available in the paragraph',
 'Their bond became a legend in the Amazon rainforest, but the specific legend is not mentioned in the paragraph']

for i in mistral:
    mistral_ans.append(i['answer'])

In [ ]:
df.to_csv("new_data.csv")